In [194]:
#Ввод данных
#МАШИНЫ
import pandas as pd
import random
import math
import time

start_time = time.time()

buyers = pd.DataFrame()

data = pd.read_csv("Car_sales.csv")

data['параметр 0'] = data['Price_in_thousands']
data['параметр 1'] = data['Horsepower']
data['параметр 2'] = data['Fuel_efficiency']

buyers.insert(0, "параметр 0", round(data['параметр 0']))
buyers.insert(1, "параметр 1", round(data['параметр 1'] * 0.1))
buyers.insert(2, "параметр 2", round(data['параметр 2'] * 0.1))

buyers = buyers.dropna()
buyers = buyers.reset_index(drop=True)

qual_amount = buyers.shape[1] - 1
print(buyers)

     параметр 0  параметр 1  параметр 2
0          22.0        14.0         3.0
1          28.0        22.0         2.0
2          42.0        21.0         2.0
3          24.0        15.0         3.0
4          34.0        20.0         2.0
..          ...         ...         ...
148        24.0        16.0         2.0
149        28.0        17.0         2.0
150        29.0        17.0         2.0
151        46.0        24.0         2.0
152        36.0        20.0         2.0

[153 rows x 3 columns]


In [195]:
#Функция снижения температуры
def decrease_temperature(initial_temperature, i):
    T = initial_temperature * 0.1 / i
    return T

#Вычисление вероятности перехода в худшее состояние
def get_probability(current, state, T):
    if (state <= current):
        P = 1
    else:
        P = math.exp(-abs(state - current)/T)
    return P

def if_transition(probability):
    value = random.randint(1, 99)/100
    if (value <= probability):
        a = 1
    else:
        a = 0
    return a

In [196]:
#Генерация состояния-кандидата
def generate_candidate(fr_name, energy):
    candidate = []
    counter = 0
    for i in range(0, qual_amount + 1):
        name1 = 'параметр ' + str(i)
        quality = random.randint(int(min(fr_name[name1][:])), int(max(fr_name[name1][:])))
        candidate.append(quality)
    return candidate

In [197]:
#Подсчёт количества покупателей при данном векторе товара
def amount_of_buyers(fr_name, vector):
    n = fr_name.shape[0]
    counter = 0
    for j in range (0, n):
        condition = (vector[0] <= fr_name['параметр 0'][j])
        for i in range(1, qual_amount + 1):
            name = 'параметр ' + str(i)
            condition &= vector[i] >= fr_name[name][j]
        if condition:
            counter += 1
    return counter

#Подсчёт прибыли
def calculate_energy(fr_name, vector):
    profit = vector[0]
    for i in range(1, qual_amount + 1):
        profit -= vector[i]
    profit = profit * amount_of_buyers(fr_name, vector)
    return profit

In [198]:
#Реализация алгоритма имиатции отжига
def simulated_annealing(fr_name, initial_temperature, end_temperature):
    state = fr_name.loc[0][:]
    
    current_energy = calculate_energy(fr_name, state)
    
    T = initial_temperature
    
    for i in range (1, 100):   
        state_candidate = generate_candidate(fr_name, current_energy)
        candidate_energy = calculate_energy(fr_name, state_candidate)
        
        if(candidate_energy > current_energy): 
            current_energy = candidate_energy
            state = state_candidate
        else:
            p = get_probability(candidate_energy, current_energy, T)
            if (if_transition(p) == 1):
                current_energy = candidate_energy
                state = state_candidate       
        
        T = decrease_temperature(initial_temperature, i)
       
        
        if(T <= end_temperature):
            break
    print(state)
    print(current_energy)
    return current_energy

simulated_annealing(buyers, 13000, 0.1)
print("--- %s seconds ---" % (time.time() - start_time))

[43, 31, 2]
180
--- 3.3321757316589355 seconds ---
